In [1]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as f
import torch.nn as nn

torch.manual_seed(0)

In [2]:
X = None

labelDict = {
    'Iris-setosa': 0,
    'Iris-versicolor': 1,
    'Iris-virginica': 2
}

def load_iris_dataset():
    with open('iris.data') as fd:
        eye = torch.eye(3).tolist()
        D = fd.read()
        D = D.strip().split('\n')
        D = [d.split(',') for d in D]
        Y = [eye[labelDict[d[4]]] for d in D]
        X = [[float(d[0]), float(d[1]), float(d[2]), float(d[3])] for d in D]
        return torch.tensor(X), torch.tensor(Y)

class Dataset:
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return X[idx], Y[idx]

In [3]:
def train(model, trainset, valset, epoch):
    optim = torch.optim.SGD(model.parameters(), lr = 0.01, weight_decay=0.001)
    train_loader = DataLoader(trainset, batch_size=8)
    best_val_acc = 0
    best_val_epoch = 0
    best_train_acc = 0
    best_train_epoch = 0
    
    for cur_epoch in range(epoch):
        train_tp = 0
        train_count = 0
        avg_loss = None
        
        for x, label in train_loader:
            optim.zero_grad()
            pred = model(x)
            loss = f.mse_loss(pred, label)
            if(avg_loss == None):
                avg_loss = loss.item()
            else:
                avg_loss += (avg_loss + loss.item()) / 2
            loss.backward()
            optim.step()
            
            with torch.no_grad():
                pred_class = pred.argmax(dim=1)
                label_class = label.argmax(dim=1)
                for p, l in zip(pred_class, label_class):
                    if(p.item() == l.item()):
                        train_tp += 1
                    train_count += 1
                    
        train_acc = train_tp / train_count
        if(train_acc > best_train_acc):
            best_train_acc = train_acc
            best_train_epoch = cur_epoch + 1
        
        val_loader = DataLoader(valset, batch_size=8)
        with torch.no_grad():
            val_tp = 0
            val_count = 0
            for x, label in val_loader:
                pred = model(x)
                pred_class = pred.argmax(dim=1)
                label_class = label.argmax(dim=1)
                
                for p, l in zip(pred_class, label_class):
                    if(p.item() == l.item()):
                        val_tp += 1
                    val_count += 1
            
            val_acc = val_tp / val_count
            if(val_acc > best_val_acc):
                best_val_acc = val_acc
                best_val_epoch = cur_epoch + 1
                
            print(f'Epoch {cur_epoch + 1}: VAL_ACC = {best_val_acc:.3f}(at {best_val_epoch}), LOSS_AVG = {avg_loss:.3f}, TRAIN_ACC = {best_train_acc:.3f} (at {best_train_epoch})', end='\r')
                

In [11]:
torch.manual_seed(0)
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(0)
        
        self.seq = nn.Sequential(
            nn.Linear(4, 5),
            nn.ReLU(),
            nn.Linear(5, 3)
        )
    
    def forward(self, x):
        return self.seq(x)

X, Y = load_iris_dataset()
dataset = Dataset(X, Y)
trainset, valset = random_split(dataset, [0.6, 0.4])

In [12]:
model = Network()
train(model, trainset, valset, 1000)